In [1]:
import folium
import pandas as pd
import re

In [2]:
# support de réprésentaion géographique des données 
# centrée autour de Lyon de coordonnées gps : [45.75, 4.85]
m = folium.Map(location=[45.75, 4.85])
m

In [3]:
# recupération des données scrappé le 8 octobre 2021 sur superimmo.com
df = pd.read_csv('../datas/superimmo/20211008/superimmo_20211008_69.csv')

# repartition des annonces par ville (code postal en fait)
df.location.value_counts()

Villeurbanne (69100)              651
Lyon 3ème (69003)                 368
Lyon 8ème (69008)                 304
Lyon 7ème (69007)                 236
Villefranche-sur-Saône (69400)    225
                                 ... 
Aigueperse (69790)                  1
Blacé (69460)                       1
Saint-Bonnet-le-Troncy (69870)      1
Lucenay (69480)                     1
Chaussan (69440)                    1
Name: location, Length: 270, dtype: int64

In [4]:
#extraction du code postaux des "location"(exemple:Marseille 8ème (13008)) via une regex
def getcp(location):
    result = re.search(r".*\((.*)\)", location)
    if result:
        return result.group(1)
    

df['Code_postal'] = df.location.apply(lambda x: getcp(str(x)))
# df.info()
# ça n'a pas marché partout
df[(df['Code_postal'].isna())][['location','Code_postal','titre']]
# parce que location n'y ait pas partout ...

# on va les retirer pour le moment mais il ne faut pas qu'on les oublie ...
df = df[-(df['Code_postal'].isna())]

# df.info()

In [5]:
# obtenir les coordonnées gps des codes postaux obtenus
# à partir du fichier pris sur : 
# https://www.data.gouv.fr/fr/datasets/base-officielle-des-codes-postaux/
coo=pd.read_csv('../datas/locations/laposte_hexasmal.csv',sep=";")
#coo.info()

# formatage
coo.Code_postal = coo.Code_postal.astype('str')

### ici je voulais faire une jointure entre les deux fichiers 
### un truc dans le genre : 
### df['gps'] = pd.merge(df, coo, how="left", on=["Code_postal"])
### mais ça ne marchait pas bien 
### 
### J'ai finalement constuit un dictionnaire clé->valeur entre les codes postaux et le coo gps
### il y surement moyen de faire mieux 

cpcoo={}
for cp,coo in zip (coo['Code_postal'].to_list(),coo['coordonnees_gps'].to_list()) :
    cpcoo[cp]=coo

def getcoo(x,cpcoo):       
    try :
        return cpcoo[x];
    except :
         return'0,0';
    

df['gps']=df.Code_postal.apply(lambda x: getcoo(str(x),cpcoo))
print(df[['Code_postal','gps']])


     Code_postal                          gps
0          69620   45.9188737019,4.5787353944
1          69100  45.7707704179,4.88845817426
2          69300  45.7974053648,4.85124857101
3          69007  45.7334098415,4.83758495703
4          69100  45.7707704179,4.88845817426
...          ...                          ...
6615       69770  45.7855981526,4.42642945537
6616       69770  45.7855981526,4.42642945537
6617       69360  45.6275993813,4.82483713363
6618       69008  45.7342034691,4.86935722305
6619       69530   45.678117475,4.75268787981

[6500 rows x 2 columns]


In [6]:
# calcul du prix au metre carrée par code postal
df = df[(df.type=='appartement')]

# seulement si on a biens des valeurs 
df = df[-(df.prix.isna())]
df = df[-(df.surface.isna())]

# formatage
# 120 500 € => 120000 par exemple
df.prix = df.prix.apply(lambda x: int(float(str(x)
                            .replace('€','')
                            .replace(' ','')
                            .replace(',','.') # certaines annonces ont des prix à virgules
                        )))
df.prix = df.prix.astype(int)

# 85,65 => 86.65 par exemple
df.surface = df.surface.apply(lambda x: str(x).replace(' ','').replace(',','.'))
df.surface = df.surface.astype(float)

# deduction du prix au metre carré
df['pm2']=df['prix']/df['surface']

print(df.groupby('Code_postal').describe()['pm2'])



             count         mean          std          min          25%  \
Code_postal                                                              
69            13.0  6021.015732  1643.685817  2873.299710  4816.901408   
69001        120.0  6410.440623  1281.688634  3673.684211  5486.725664   
69002        144.0  6930.230519  1418.994833  4554.140127  6004.460745   
69003        291.0  5504.216494  1145.284107  3194.845361  4731.800766   
69004        141.0  6050.381435  1412.318410  2884.057971  5169.063350   
...            ...          ...          ...          ...          ...   
69870          5.0  1003.767248   313.742634   753.246753   900.000000   
69890          7.0  5171.545301   569.842776  4233.829831  4826.486919   
69930          1.0  1964.285714          NaN  1964.285714  1964.285714   
69960          9.0  3309.211540   910.304955  2261.904762  2500.000000   
69970          3.0  3561.401933    74.909863  3474.903475  3539.777319   

                     50%          75%

In [7]:
# recensement des données à placer sur la carte regroupées par code postal
dfByCp = df.groupby('Code_postal').agg({'pm2' : ['mean'],     # prix au mettre moyen des annonces
                                        'gps':['min'],        # coordonnées gps (qui sont normalement toutes les mêmes)
                                        'location':['min']    # nom de lieu correspondant au code postal
                                       })
print(dfByCp)

print(dfByCp['pm2']['mean'].min())

                     pm2                          gps  \
                    mean                          min   
Code_postal                                             
69           6021.015732                          0,0   
69001        6410.440623  45.7699284397,4.82922464978   
69002        6930.230519  45.7492560394,4.82617032549   
69003        5504.216494  45.7533552486,4.86918522013   
69004        6050.381435  45.7786867581,4.82396112689   
...                  ...                          ...   
69870        1003.767248   46.0832387842,4.4623284164   
69890        5171.545301   45.8104377668,4.7123892929   
69930        1964.285714   45.756189691,4.42060424905   
69960        3309.211540  45.6680498711,4.90843949215   
69970        3561.401933  45.6230631164,4.90753751863   

                                        location  
                                             min  
Code_postal                                       
69                                     Lyon (69)

In [8]:
# utilisation d'un dégradé de couleur 
# pour indentifier visuellement les prix au mettre carré
# obtenue grace au site : https://colordesigner.io/gradient-generator

gradient40 = ['#fafa6e',
'#faf568',
'#f9f063',
'#f9eb5d',
'#f8e658',
'#f8e153',
'#f7dc4d',
'#f7d748',
'#f6d243',
'#f6cd3f',
'#f5c83a',
'#f4c335',
'#f4bd30',
'#f3b82c',
'#f2b327',
'#f1ae23',
'#f0a91f',
'#efa41b',
'#ee9e16',
'#ed9912',
'#ec940e',
'#eb8e0a',
'#ea8906',
'#e88303',
'#e77e01',
'#e67800',
'#e47300',
'#e36d00',
'#e16700',
'#df6100',
'#de5b00',
'#dc5501',
'#da4f02',
'#d84804',
'#d64105',
'#d43a07',
'#d23209',
'#d0290b',
'#cd1e0d',
'#cb0f0f']

# colorimétrie faite en fractionnant les valeurs 
# sur une échelle de 40 valeurs partagées 
# entre le prix au mettre carré minimum et maximum
mrg=1 # marge pour éviter les sorties mémoires "out of index"
print("Répartion géographique des prix au mettre carré")
print("des appartements des annonces de superimmo.com")
print("du Rhone fait le 8 octobre 2021")
print("min : "+str(dfByCp['pm2']['mean'].min()))
print("max : "+str(dfByCp['pm2']['mean'].max()))

for gps,location,pm2 in zip (dfByCp['gps']['min'].to_list(),
                             dfByCp['location']['min'].to_list(),
                             dfByCp['pm2']['mean'].to_list()) :
    folium.CircleMarker(
    location=gps.split(','),
    radius=10,
    popup=location+" : "+str(pm2),
    color=gradient40[int((pm2-dfByCp['pm2']['mean'].min()-mrg)/(dfByCp['pm2']['mean'].max()+100-dfByCp['pm2']['mean'].min()-mrg)*40)], 
    fill=True,
    fill_color=gradient40[int((pm2-dfByCp['pm2']['mean'].min()-mrg)/(dfByCp['pm2']['mean'].max()+100-dfByCp['pm2']['mean'].min()-mrg)*40)],
    ).add_to(m)    
m 

Répartion géographique des prix au mettre carré
des appartements des annonces de superimmo.com
du Rhone fait le 8 octobre 2021
min : 700.0
max : 6930.230518784218
